<img src="../docs/images/DSPy8.png" alt="DSPy7 Image" height="120"/>

### Migrating from DSPy 2.4 to 2.5

**DSPy 2.5** focuses on improving the _pre-optimization_ developer experience, i.e. making it easier to obtain higher-quality outputs from various LMs _out of the box_ prior to setting up a DSPy optimizer.

### tl;dr Just set up the LM differently at the start.

For most applications, you will only need to change 1-2 lines.

In [ ]:
import dspy

lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

The rest of your code can remain unchanged. However, make sure to experiment with your modules. Improvements to **Adapters** will affect the underlying prompts, before and after optimization, and will enforce stricter validation for LM outputs.

If you want to dive deeper, here's are some new things and things to keep in mind when upgrading.

1. **Use the `dspy.LM` class for setting up language models.**
2. **Invoke LMs in the same way as before.**
3. **This internally uses new DSPy `Adapters`, leading to better prompting out of the box.**
4. **Advanced: If needed, you can set up your own Adapter for LMs.**

### 1) Use the `dspy.LM` class for setting up language models.

Earlier versions of DSPy involved tens of clients for different LM providers, e.g. `dspy.OpenAI`, `dspy.GoogleVertexAI`, and `dspy.HFClientTGI`, etc. These are now deprecated and will be removed in DSPy 2.6.

Instead, use `dspy.LM` to access any LM endpoint for local and remote models. This relies on [LiteLLM](https://github.com/BerriAI/litellm) to translate the different client APIs into an OpenAI-compatible interface.

Any [provider supported in LiteLLM](https://docs.litellm.ai/docs/providers) should work with `dspy.LM`.

In [2]:
# OpenAI, which can authenticate via OPENAI_API_KEY.
lm = dspy.LM('openai/gpt-4o-mini')

# Anthropic, which can authenticate via ANTHROPIC_API_KEY.
anthropic_lm = dspy.LM('anthropic/claude-3-opus-20240229')

# You can also pass auth information directly.
anthropic_lm = dspy.LM('anthropic/claude-3-opus-20240229', api_key="..", api_base="..")

# Cohere, which can authenticate via COHERE_API_KEY.
cohere_lm = dspy.LM('cohere/command-nightly')

# Databricks, which can authenticate via DATABRICKS_API_KEY & DATABRICKS_API_BASE (or automatically on a DB workspace).
databricks_llama3 = dspy.LM('databricks/databricks-meta-llama-3-1-70b-instruct')

# or many others.

Any OpenAI-compatible endpoint is easy to set up with an `openai/` prefix as well. This works great for open LMs from HuggingFace hosted locally with SGLang, VLLM, or HF Text-Generation-Inference.

In [3]:
sglang_port = 7501
sglang_url = f"http://localhost:{sglang_port}/v1"
sglang_llama = dspy.LM("openai/meta-llama/Meta-Llama-3-8B-Instruct", api_base=sglang_url)

# You could also use text mode, in which the prompts are *not* formatted as messages.
sglang_llama_text = dspy.LM("openai/meta-llama/Meta-Llama-3-8B-Instruct", api_base=sglang_url, model_type='text')

#### Configuring LM attributes

For any LM, you can configure any of the following attributes at initialization or per call.

In [4]:
gpt_4o_mini = dspy.LM('openai/gpt-4o-mini', temperature=0.9, max_tokens=3000, stop=None, cache=False)

Caching.

Old clients still exist but you'll get a deprecation warning. They will be removed in DSPy 2.6.

### 2) Invoke LMs in the same way as before.

Invoking the LM directly with a prompt is the same as before.

In [5]:
lm("What is 2+2?", temperature=0.9)

['2 + 2 equals 4.']

For chat LMs, you can pass a list of messages.

In [6]:
lm(messages=[{"role": "system", "content": "You are a helpful assistant."},
             {"role": "user", "content": "What is 2+2?"}])

['2 + 2 equals 4.']

Using DSPy modules is the same as before.

In [7]:
module = dspy.ChainOfThought('question -> answer')
module(question="What is 2+2?")

Prediction(
    reasoning='To solve the equation 2 + 2, we simply add the two numbers together. The number 2 represents a quantity, and when we add another 2 to it, we are combining these quantities. Therefore, 2 + 2 equals 4.',
    answer='4'
)

Every LM object maintains the history of its interactions, including inputs, outputs, token usage (and $$$ cost), and metadata.

In [8]:
len(lm.history)  # e.g., 3 calls to the LM

lm.history[-1].keys()  # access the last call to the LM, with all metadata

dict_keys(['prompt', 'messages', 'kwargs', 'response', 'outputs', 'usage', 'cost'])

### 3) This internally uses new DSPy `Adapters`, leading to better prompting out of the box.

Prompt optimizers in DSPy generate and tune the _instructions_ or the _examples_ in the prompts corresponding to your Signatures. DSPy 2.5 introduces **Adapters** as a layer between Signatures and LMs, responsible for formatting these pieces (Signature I/O fields, instructions, and examples) as well as generating and parsing the outputs. In DSPy 2.5, the default Adapters are now more natively aware of chat LMs and are responsible for enforcing types, building on earlier experimental features from `TypedPredictor` and `configure(experimental=True)`. In our experience, this change tends to deliver more consistent pre-optimization quality from various LMs, especially for sophisticated Signatures.

In [9]:
fact_checking = dspy.ChainOfThought('claims -> verdicts: list[bool]')
fact_checking(claims=["Python was released in 1991.", "Python is a compiled language."])

Prediction(
    reasoning='The first claim states that "Python was released in 1991," which is true. Python was indeed first released by Guido van Rossum in February 1991. The second claim states that "Python is a compiled language." This is false; Python is primarily an interpreted language, although it can be compiled to bytecode, it is not considered a compiled language in the traditional sense like C or Java.',
    verdicts=[True, False]
)

We can as usual inspect the last prompt (and output) from an LM call.

In [10]:
dspy.inspect_history()





System message:

Your input fields are:
1. `claims` (str)

Your output fields are:
1. `reasoning` (str)
2. `verdicts` (list[bool])

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## claims ## ]]
{claims}

[[ ## reasoning ## ]]
{reasoning}

[[ ## verdicts ## ]]
{verdicts}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `claims`, produce the fields `verdicts`.


User message:

[[ ## claims ## ]]
[1] «Python was released in 1991.»
[2] «Python is a compiled language.»

Respond with the corresponding output fields, starting with the field `reasoning`, then `verdicts`, and then ending with the marker for `completed`.


Response:

[[ ## reasoning ## ]]
The first claim states that "Python was released in 1991," which is true. Python was indeed first released by Guido van Rossum in February 1991. The second claim states that "Python is a compiled language." This is false; Python is prim

#### Stricter validation for LM outputs.

Previously, only `TypedPredictor`s had strict validation for LM outputs. This is now in all DSPy modules.

In [11]:
# a module with insufficient tokens to format the output
bad_module = dspy.ChainOfThought('input -> output: list[str]', max_tokens=5)

try:
    bad_module(input='oops')
except ValueError as e:
    print("When parsing the output:", e)

When parsing the output: Expected dict_keys(['reasoning', 'output']) but got dict_keys(['reasoning'])


We are currently considering a "soft failure" mode, which fills failed fields with `None`s for cases that don't require explicit exception handling. We are also working on more accessible exceptions and error messages in future 2.5.* releases.

### 4) Advanced: If needed, you can set up your own Adapter for LMs.

Adapters are the first step towards DSPy optimizers that tune the templates themselves. They also make it much easier for you to adapt the templates to your needs programatically, though that remains discouraged for most applications since such effort is often best spent on a program's control flow or optimization. At a minimum, an Adapter defines a method `format(signature, demos, inputs)` that prepares a list of messages (or a prompt string) and a method `parse(signature, completion)` that extract a `dict` of output fields. You can also overwrite `__call__(lm, lm_kwargs, signature, demos, inputs)` to refine the generation or decoding logic for your signatures.

In [12]:
dspy.configure(adapter=dspy.ChatAdapter())  # ChatAdapter is the default adapter, but you can override it.

### 5) What's next?

In the next few weeks, we will release into DSPy 2.5 a number of new updates, including: (1) optimizable adapters, smarter retries, better exceptions, support for images, support for multi-turn signatures, improved finetuning and assertions, concurrent with other ongoing efforts from the [DSPy Roadmap](https://github.com/stanfordnlp/dspy/blob/main/docs/docs/roadmap.md), which include more powerful optimizers, human-in-the-loop processes, etc.